In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 --executor-memory 3g --driver-memory 2g pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))

In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

conf = SparkConf()
conf.set("spark.app.name", "PM app") 

spark = SparkSession.builder.config(conf=conf).appName("PM app").getOrCreate()

In [3]:
spark

In [13]:
!hdfs dfs -get /labs/slaba03/laba03_test.csv #laba03_views_programmes.csv #laba03_train.csv #laba03_test.csv #laba03_items.csv

get: `laba03_test.csv': File exists


In [5]:
! more laba03_items.csv

item_id	channel_id	datetime_availability_start	datetime_availability_st
op	datetime_show_start	datetime_show_stop	content_type	title	
year	genres	region_id
65667		1970-01-01T00:00:00Z	2018-01-01T00:00:00Z			
1	на пробах только девушки (all girl auditions)	2013.0	Эротика	
65669		1970-01-01T00:00:00Z	2018-01-01T00:00:00Z			
1	скуби ду: эротическая пародия (scooby doo: a xxx parody)	2011.0	
Эротика	
65668		1970-01-01T00:00:00Z	2018-01-01T00:00:00Z			
1	горячие девочки для горячих девочек (hot babes 4 hot babes)	2011.0	
Эротика	
65671		1970-01-01T00:00:00Z	2018-01-01T00:00:00Z			
1	соблазнительницы женатых мужчин (top heavy homewreckers)	2011.0	
Эротика	
65670		1970-01-01T00:00:00Z	2018-01-01T00:00:00Z			
1	секретные секс-материалы ii: темная секс пародия (the sex files ii: a da
rk xxx parody)	2010.0	Эротика	
65809		1970-01-01T00:00:00Z	2099-12-31T21:00:00Z			
1	все о мужчинах	2016.0	Комедии	
65810		1970-01-01T00:00:00Z	2099-12-31T21:00:00Z			
1	8 лучших свиданий (сурдоперевод)	2016.0	Коме

In [1]:
# ! more laba03_views_programmes.csv

In [9]:
from pyspark.sql.types import LongType
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [15]:
test_df = spark.read\
          .format("csv")\
          .option("header","true")\
          .option("sep", ",")\
          .load("/labs/slaba03/laba03_test.csv")

In [16]:
test_df.show(10)

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
|   1654|  94814|    null|
|   1654|  93629|    null|
|   1654|   9980|    null|
|   1654|  95099|    null|
|   1654|  11265|    null|
|   1654|  88896|    null|
|   1654|  67740|    null|
|   1654|  74271|    null|
|   1654|  99871|    null|
|   1654|  78570|    null|
+-------+-------+--------+
only showing top 10 rows



In [17]:
test_df.count()

2156840

In [18]:
programmes_schema = StructType(fields=[
    StructField("user_id", IntegerType()),
    StructField("item_id", IntegerType()),
    StructField("ts_start", LongType()),
    StructField("ts_end", LongType()),
    StructField("item_type", StringType())    
])

In [20]:
programmes = spark.read\
          .format("csv")\
          .option("header","true")\
          .option("sep", ",")\
          .load("/labs/slaba03/laba03_views_programmes.csv")

In [21]:
programmes.show(10)

+-------+-------+----------+----------+---------+
|user_id|item_id|  ts_start|    ts_end|item_type|
+-------+-------+----------+----------+---------+
|      0|7101053|1491409931|1491411600|     live|
|      0|7101054|1491412481|1491451571|     live|
|      0|7101054|1491411640|1491412481|     live|
|      0|6184414|1486191290|1486191640|     live|
|    257|4436877|1490628499|1490630256|     live|
|   1654|7489015|1493434801|1493435401|     live|
|   1654|7489023|1493444101|1493445601|     live|
|   1654|6617053|1489186156|1489200834|     live|
|   1654|6438693|1487840070|1487840433|     live|
|   1654|6526859|1488705452|1488706154|     live|
+-------+-------+----------+----------+---------+
only showing top 10 rows



In [22]:
programmes.createOrReplaceTempView("programmes_ql")

In [23]:
query = """
SELECT 
  user_id
, item_id
, item_type
, to_timestamp(from_unixtime(ts_start,'yyyy-MM-dd HH:mm:ss')) as ts_start
, to_timestamp(from_unixtime(ts_end,'yyyy-MM-dd HH:mm:ss')) as ts_end
--, datediff(to_timestamp(from_unixtime(ts_start,'yyyy-MM-dd HH:mm:ss')),to_timestamp(from_unixtime(ts_end,'yyyy-MM-dd HH:mm:ss'))) as diff
, round((ts_end - ts_start)/60) as diff_sec
--,item_type
FROM programmes_ql
--where 'user_id' = 1654
"""

In [24]:
spark.sql(query).show(10)

+-------+-------+---------+-------------------+-------------------+--------+
|user_id|item_id|item_type|           ts_start|             ts_end|diff_sec|
+-------+-------+---------+-------------------+-------------------+--------+
|      0|7101053|     live|2017-04-05 19:32:11|2017-04-05 20:00:00|    28.0|
|      0|7101054|     live|2017-04-05 20:14:41|2017-04-06 07:06:11|   652.0|
|      0|7101054|     live|2017-04-05 20:00:40|2017-04-05 20:14:41|    14.0|
|      0|6184414|     live|2017-02-04 09:54:50|2017-02-04 10:00:40|     6.0|
|    257|4436877|     live|2017-03-27 18:28:19|2017-03-27 18:57:36|    29.0|
|   1654|7489015|     live|2017-04-29 06:00:01|2017-04-29 06:10:01|    10.0|
|   1654|7489023|     live|2017-04-29 08:35:01|2017-04-29 09:00:01|    25.0|
|   1654|6617053|     live|2017-03-11 01:49:16|2017-03-11 05:53:54|   245.0|
|   1654|6438693|     live|2017-02-23 11:54:30|2017-02-23 12:00:33|     6.0|
|   1654|6526859|     live|2017-03-05 12:17:32|2017-03-05 12:29:14|    12.0|

In [27]:
items = spark.read\
          .format("csv")\
          .option("header","true")\
          .option("sep", "\t")\
          .load("/labs/slaba03/laba03_items.csv")

In [28]:
items.show(1,False,False)

+-------+----------+---------------------------+--------------------------+-------------------+------------------+------------+---------------------------------------------+------+-------+---------+
|item_id|channel_id|datetime_availability_start|datetime_availability_stop|datetime_show_start|datetime_show_stop|content_type|title                                        |year  |genres |region_id|
+-------+----------+---------------------------+--------------------------+-------------------+------------------+------------+---------------------------------------------+------+-------+---------+
|65667  |null      |1970-01-01T00:00:00Z       |2018-01-01T00:00:00Z      |null               |null              |1           |на пробах только девушки (all girl auditions)|2013.0|Эротика|null     |
+-------+----------+---------------------------+--------------------------+-------------------+------------------+------------+---------------------------------------------+------+-------+---------+
only 

In [29]:
items.createOrReplaceTempView("items_sql")

In [30]:
q0 = """
SELECT 
 count(distinct item_id) as count_paid_channels
FROM items_sql
where content_type ==1

"""

In [31]:
spark.sql(q0).show()

+-------------------+
|count_paid_channels|
+-------------------+
|               3704|
+-------------------+



In [32]:
q = """
SELECT 
  item_id
, channel_id
, datetime_availability_start as av_start
, datetime_availability_stop as av_stop
, datetime_show_start as show_st
, datetime_show_stop as show_st
, content_type
, title
, year
, genres
,split(genres,',')[0] as first_genre
, region_id
FROM items_sql
where content_type ==1

--user_id = 1654
"""

In [33]:
items2 = spark.sql(q)
items2.show(1,False)

+-------+----------+--------------------+--------------------+-------+-------+------------+---------------------------------------------+------+-------+-----------+---------+
|item_id|channel_id|av_start            |av_stop             |show_st|show_st|content_type|title                                        |year  |genres |first_genre|region_id|
+-------+----------+--------------------+--------------------+-------+-------+------------+---------------------------------------------+------+-------+-----------+---------+
|65667  |null      |1970-01-01T00:00:00Z|2018-01-01T00:00:00Z|null   |null   |1           |на пробах только девушки (all girl auditions)|2013.0|Эротика|Эротика    |null     |
+-------+----------+--------------------+--------------------+-------+-------+------------+---------------------------------------------+------+-------+-----------+---------+
only showing top 1 row



In [34]:
items3 = items2.select('item_id','first_genre').na.drop()
items3.show(10)

+-------+-----------+
|item_id|first_genre|
+-------+-----------+
|  65667|    Эротика|
|  65669|    Эротика|
|  65668|    Эротика|
|  65671|    Эротика|
|  65670|    Эротика|
|  65809|    Комедии|
|  65810|    Комедии|
|    326|      Ужасы|
|    336|      Ужасы|
|    357|    Комедии|
+-------+-----------+
only showing top 10 rows



In [35]:
items3.count()

3671

In [36]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer

In [37]:
stringIndexer = StringIndexer(inputCol="first_genre", outputCol="genre_Index")
model = stringIndexer.fit(items3)
indexed = model.transform(items3)

In [38]:
indexed.show(10)

+-------+-----------+-----------+
|item_id|first_genre|genre_Index|
+-------+-----------+-----------+
|  65667|    Эротика|        9.0|
|  65669|    Эротика|        9.0|
|  65668|    Эротика|        9.0|
|  65671|    Эротика|        9.0|
|  65670|    Эротика|        9.0|
|  65809|    Комедии|        0.0|
|  65810|    Комедии|        0.0|
|    326|      Ужасы|        3.0|
|    336|      Ужасы|        3.0|
|    357|    Комедии|        0.0|
+-------+-----------+-----------+
only showing top 10 rows



In [39]:
encoder = OneHotEncoder(inputCol="genre_Index", outputCol="genre_vector")
item_genre_vector = encoder.transform(indexed)


In [40]:
item_genre_vector.show()

+-------+-----------+-----------+---------------+
|item_id|first_genre|genre_Index|   genre_vector|
+-------+-----------+-----------+---------------+
|  65667|    Эротика|        9.0| (63,[9],[1.0])|
|  65669|    Эротика|        9.0| (63,[9],[1.0])|
|  65668|    Эротика|        9.0| (63,[9],[1.0])|
|  65671|    Эротика|        9.0| (63,[9],[1.0])|
|  65670|    Эротика|        9.0| (63,[9],[1.0])|
|  65809|    Комедии|        0.0| (63,[0],[1.0])|
|  65810|    Комедии|        0.0| (63,[0],[1.0])|
|    326|      Ужасы|        3.0| (63,[3],[1.0])|
|    336|      Ужасы|        3.0| (63,[3],[1.0])|
|    357|    Комедии|        0.0| (63,[0],[1.0])|
|    396|  Детективы|        5.0| (63,[5],[1.0])|
|    400| Фантастика|       14.0|(63,[14],[1.0])|
|    423|  Детективы|        5.0| (63,[5],[1.0])|
|    430|  Детективы|        5.0| (63,[5],[1.0])|
|    449|      Ужасы|        3.0| (63,[3],[1.0])|
|    453|    Комедии|        0.0| (63,[0],[1.0])|
|    478|    Военные|        7.0| (63,[7],[1.0])|


In [41]:
spark.sql(q).count() #show(10,False)

3704

In [42]:
q2 = """
SELECT   
 --genres
 distinct
  item_id
, split(genres,',')[0] as first_genre
 --, patindex(',',genres)
FROM items_sql
where content_type ==1
"""

In [43]:
item_genres = spark.sql(q2)  #.show(1000,False)
item_genres = item_genres.na.fill('-')
item_genres.show(10)

+-------+-----------+
|item_id|first_genre|
+-------+-----------+
|   4419|      Ужасы|
|   5084|    Боевики|
|   8635|Мультфильмы|
|   9040|      Драмы|
|  11040|    Комедия|
|  11071|    Комедии|
|  66457|    Военные|
|  66700|Мистические|
|  72327|    Комедии|
|  73568|      Ужасы|
+-------+-----------+
only showing top 10 rows



In [44]:
item_genres.count()

3704

In [45]:
q3 = """
SELECT 
 distinct item_id, split(genres,',') as all_genres
--, region_id
FROM items_sql

where content_type ==1

"""

In [46]:
genres = spark.sql(q3).na.drop()  #.show(1000,False)

In [47]:
genres.orderBy('all_genres').show()

+-------+--------------------+
|item_id|          all_genres|
+-------+--------------------+
|   9418|[Анимация, Для вз...|
|   1789|[Анимация, Для вз...|
|  97806|[Артхаус, Военные...|
|  95573|[Артхаус, Военные...|
|  71843|[Артхаус, Военные...|
|  98235|[Артхаус, Детекти...|
|  74555|[Артхаус, Драмы, ...|
|  74272|[Артхаус, Драмы, ...|
|  74478|[Артхаус, Драмы, ...|
| 100735|[Артхаус, Драмы, ...|
|  76264|[Артхаус, Драмы, ...|
| 100715|[Артхаус, Драмы, ...|
|  74425|[Артхаус, Драмы, ...|
|  99471|[Артхаус, Комедии...|
|  98274|[Артхаус, Комедии...|
|  73981|[Артхаус, Коротко...|
|  95278|[Артхаус, Мелодра...|
|  95416|[Артхаус, Мистиче...|
|  67318|[Артхаус, Мистиче...|
|  87167|[Артхаус, Приключ...|
+-------+--------------------+
only showing top 20 rows



In [49]:
from pyspark.ml.feature import CountVectorizer
cv = CountVectorizer()
cv.setInputCol("all_genres")
cv.setOutputCol("all_genres_vectors")

CountVectorizer_04233f121c9f

In [50]:
model = cv.fit(genres)

In [51]:
model.transform(genres).count()

3671

In [52]:
all_genres_vectors = model.transform(genres)
all_genres_vectors.where("item_id==72214").show()

+-------+----------------+--------------------+
|item_id|      all_genres|  all_genres_vectors|
+-------+----------------+--------------------+
|  72214|[Драмы, Русские]|(83,[1,4],[1.0,1.0])|
+-------+----------------+--------------------+



In [53]:
group_genres_df = item_genres.groupBy("first_genre")\
                .agg(F.count("first_genre").alias("count_first_genres"))\
                .orderBy("count_first_genres", ascending=False)
group_genres_df.show(20)                            #where('first_genre<>"General"')

+--------------------+------------------+
|         first_genre|count_first_genres|
+--------------------+------------------+
|             Комедии|               597|
|               Драмы|               362|
|         Приключения|               341|
|               Ужасы|               336|
|            Триллеры|               267|
|           Детективы|               256|
|         Мультфильмы|               241|
|             Военные|               172|
| Русские мультфильмы|               169|
|             Эротика|               121|
|            Семейные|                72|
|         Мистические|                70|
|           Мелодрамы|                67|
|      Полнометражные|                58|
|          Фантастика|                53|
|             Боевики|                48|
|      Документальные|                46|
|Западные мультфильмы|                42|
|             Русские|                36|
|                   -|                33|
+--------------------+------------

In [54]:
items2.createOrReplaceTempView("items2_sql")

In [55]:
q3 = """
SELECT   distinct
 first_genre
 , count(item_id) over (partition by first_genre) as count_genres
FROM items2_sql
order by count_genres desc
"""

In [56]:
spark.sql(q3).show()

+--------------------+------------+
|         first_genre|count_genres|
+--------------------+------------+
|             Комедии|         597|
|               Драмы|         362|
|         Приключения|         341|
|               Ужасы|         336|
|            Триллеры|         267|
|           Детективы|         256|
|         Мультфильмы|         241|
|             Военные|         172|
| Русские мультфильмы|         169|
|             Эротика|         121|
|            Семейные|          72|
|         Мистические|          70|
|           Мелодрамы|          67|
|      Полнометражные|          58|
|          Фантастика|          53|
|             Боевики|          48|
|      Документальные|          46|
|Западные мультфильмы|          42|
|             Русские|          36|
|                null|          33|
+--------------------+------------+
only showing top 20 rows



In [238]:
train = spark.read\
          .format("csv")\
          .option("header","true")\
          .option("sep", ",")\
          .load("/labs/slaba03/laba03_train.csv")

In [239]:
train.count()

5032624

In [240]:
train.where("user_id ==747028").show(10)

+-------+-------+--------+
|user_id|item_id|purchase|
+-------+-------+--------+
| 747028|  72214|       1|
| 747028|  94137|       0|
| 747028|  10880|       0|
| 747028| 100306|       0|
| 747028|  99770|       0|
| 747028|  94632|       0|
| 747028|  66467|       0|
| 747028|  82917|       1|
| 747028|  99434|       0|
| 747028|  99815|       0|
+-------+-------+--------+
only showing top 10 rows



In [241]:
train.createOrReplaceTempView("train_sql")

In [242]:
query = """
select *
, sum_over_user_id/user_buy_frequency as one_hot_user_id_solvency
from (

SELECT distinct 
  user_id
, count(user_id) over (partition by user_id) as user_buy_frequency
, sum(purchase) over (partition by user_id) as sum_over_user_id
FROM train_sql
)
order by sum_over_user_id desc
--where user_id = 1654
"""

In [243]:
spark.sql(query).count()

1941

In [244]:
one_hot_user_id_solvency = spark.sql(query)
one_hot_user_id_solvency.show(20)

+-------+------------------+----------------+------------------------+
|user_id|user_buy_frequency|sum_over_user_id|one_hot_user_id_solvency|
+-------+------------------+----------------+------------------------+
| 747028|              2632|           490.0|     0.18617021276595744|
| 588378|              2590|           138.0|     0.05328185328185328|
| 865152|              2615|           116.0|    0.044359464627151055|
| 845568|              2558|            97.0|     0.03792025019546521|
| 857251|              2641|            95.0|     0.03597122302158273|
| 910185|              2610|            82.0|    0.031417624521072794|
| 899088|              2600|            79.0|    0.030384615384615385|
| 822322|              2567|            79.0|     0.03077522399688352|
| 892523|              2603|            76.0|    0.029197080291970802|
| 898757|              2599|            75.0|    0.028857252789534438|
| 754230|              2611|            72.0|    0.027575641516660282|
| 9168

In [245]:
one_hot_user_id_solvency.count()

1941

In [246]:
import pyspark.sql.functions as f

In [247]:
query2 = """
select *,sum_over_item_id/count_item_id as one_hot_item_id_popularity 
from (

SELECT distinct item_id
, count(item_id) over (partition by item_id) as count_item_id
,sum(purchase) over (partition by item_id) as sum_over_item_id
FROM train_sql
)
order by sum_over_item_id desc

--where user_id = 1654
"""

In [248]:
spark.sql(query2).count()

3704

In [249]:
one_hot_item_id_popularity = spark.sql(query2)
one_hot_item_id_popularity.where("item_id==72214").show(20)

+-------+-------------+----------------+--------------------------+
|item_id|count_item_id|sum_over_item_id|one_hot_item_id_popularity|
+-------+-------------+----------------+--------------------------+
|  72214|         1401|             2.0|      0.001427551748750...|
+-------+-------------+----------------+--------------------------+



In [250]:
items.filter("item_id==72214").show(1,False,False)

+-------+----------+---------------------------+--------------------------+-------------------+------------------+------------+-----+------+-------------+---------+
|item_id|channel_id|datetime_availability_start|datetime_availability_stop|datetime_show_start|datetime_show_stop|content_type|title|year  |genres       |region_id|
+-------+----------+---------------------------+--------------------------+-------------------+------------------+------------+-----+------+-------------+---------+
|72214  |null      |1970-01-01T00:00:00Z       |2099-12-31T21:00:00Z      |null               |null              |1           |мажор|2014.0|Драмы,Русские|null     |
+-------+----------+---------------------------+--------------------------+-------------------+------------------+------------+-----+------+-------------+---------+



In [251]:
train_full_features = train.join(f.broadcast(one_hot_user_id_solvency), on="user_id", how="inner")\
.join(f.broadcast(one_hot_item_id_popularity), on="item_id", how="inner")\
.join(f.broadcast(all_genres_vectors), on="item_id", how="inner")\
.join(f.broadcast(item_genre_vector), on="item_id", how="inner")
train_full_features.count()

4987875

In [252]:
from pyspark.sql.types import DoubleType
from pyspark.sql.types import IntegerType

In [253]:
# join1.orderBy("item_id").show()
select_features = train_full_features.select('item_id','user_id','one_hot_user_id_solvency','one_hot_item_id_popularity','all_genres_vectors','genre_vector','purchase')\
.withColumn("item_user",f.concat(f.col('item_id'), f.col('user_id')))

select_features = select_features.withColumn("item_id", select_features["item_id"].cast(IntegerType()))
select_features = select_features.withColumn("user_id", select_features["user_id"].cast(IntegerType()))
select_features = select_features.withColumn("purchase", select_features["purchase"].cast(IntegerType()))
select_features = select_features.withColumn("item_user", select_features["item_user"].cast(DoubleType()))

select_features.where("user_id=747028").show(1)
# train_full_features = train_full_features.withColumn("user_id2", train_full_features["user_id"].cast(IntegerType))

+-------+-------+------------------------+--------------------------+--------------------+--------------+--------+---------------+
|item_id|user_id|one_hot_user_id_solvency|one_hot_item_id_popularity|  all_genres_vectors|  genre_vector|purchase|      item_user|
+-------+-------+------------------------+--------------------------+--------------------+--------------+--------+---------------+
|  72214| 747028|     0.18617021276595744|      0.001427551748750...|(83,[1,4],[1.0,1.0])|(63,[1],[1.0])|       1|7.2214747028E10|
+-------+-------+------------------------+--------------------------+--------------------+--------------+--------+---------------+
only showing top 1 row



In [254]:
select_features.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- one_hot_user_id_solvency: double (nullable = true)
 |-- one_hot_item_id_popularity: double (nullable = true)
 |-- all_genres_vectors: vector (nullable = true)
 |-- genre_vector: vector (nullable = true)
 |-- purchase: integer (nullable = true)
 |-- item_user: double (nullable = true)



In [255]:
select_features.where("purchase==1").count()

10576

In [256]:
select_features.where("purchase==0").count()

4977299

In [257]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [314]:
assembler = VectorAssembler(inputCols=['user_id','item_id',"one_hot_user_id_solvency", "one_hot_item_id_popularity","all_genres_vectors","genre_vector",'purchase'],outputCol="features") # "all_genres_vectors",,"genre_vector"

In [315]:
output = assembler.transform(select_features)

In [317]:
train1, test1 = output.randomSplit([0.8, 0.2])

In [320]:
gbt = GBTClassifier(labelCol="purchase", featuresCol="features", maxIter=20)

In [321]:
model = gbt.fit(train1)

In [322]:
predictions = model.transform(test1)

In [325]:
predictions.where('user_id==1654').select('prediction', 'probability').show(25,False)

+----------+-----------------------------------------+
|prediction|probability                              |
+----------+-----------------------------------------+
|0.0       |[0.9563534785730002,0.04364652142699976] |
|0.0       |[0.9563534785729898,0.0436465214270102]  |
|0.0       |[0.9563534785729929,0.04364652142700709] |
|0.0       |[0.9563534785730108,0.043646521426989215]|
|0.0       |[0.9563534785730179,0.04364652142698211] |
|0.0       |[0.9563534785730108,0.043646521426989215]|
|0.0       |[0.9563534785730216,0.043646521426978446]|
|0.0       |[0.9563534785730383,0.04364652142696168] |
|0.0       |[0.9563534785730179,0.04364652142698211] |
|0.0       |[0.9563534785730329,0.04364652142696712] |
|0.0       |[0.9563534785730223,0.04364652142697767] |
|0.0       |[0.9563534785729694,0.04364652142703063] |
|0.0       |[0.9563534785730228,0.043646521426977225]|
|0.0       |[0.9563534785730179,0.04364652142698211] |
|0.0       |[0.9563534785730412,0.043646521426958795]|
|0.0      

In [326]:
gbt_eval = BinaryClassificationEvaluator(rawPredictionCol="probability",labelCol="purchase")

In [327]:
gbt_AUC  = gbt_eval.evaluate(predictions)
print("AUC = %.4f" % gbt_AUC)

AUC = 1.0000


In [110]:
predictions.show(1)

+-------+-------+------------------------+--------------------------+--------------------+--------------+--------+------------+--------------------+--------------------+--------------------+----------+
|item_id|user_id|one_hot_user_id_solvency|one_hot_item_id_popularity|  all_genres_vectors|  genre_vector|purchase|   item_user|            features|       rawPrediction|         probability|prediction|
+-------+-------+------------------------+--------------------------+--------------------+--------------+--------+------------+--------------------+--------------------+--------------------+----------+
|    326| 613775|    3.894080996884735E-4|      7.320644216691069E-4|(83,[0,1,3,10,12]...|(63,[3],[1.0])|       0|3.26613775E8|(151,[0,1,2,3,4,5...|[1.32556415050051...|[0.93408049868664...|       0.0|
+-------+-------+------------------------+--------------------------+--------------------+--------------+--------+------------+--------------------+--------------------+--------------------+--

In [328]:
evaluator = MulticlassClassificationEvaluator(labelCol="purchase", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))


Test Error = 0


In [112]:
test_full_features = test_df.join(f.broadcast(one_hot_user_id_solvency), on="user_id", how="inner")\
.join(f.broadcast(one_hot_item_id_popularity), on="item_id", how="inner")\
.join(f.broadcast(all_genres_vectors), on="item_id", how="inner")\
.join(f.broadcast(item_genre_vector), on="item_id", how="inner")
test_full_features.count()

2137536

In [113]:
test_full_features_FULL = test_df.join(f.broadcast(one_hot_user_id_solvency), on="user_id", how="left")\
.join(f.broadcast(one_hot_item_id_popularity), on="item_id", how="left")\
.join(f.broadcast(all_genres_vectors), on="item_id", how="left")\
.join(f.broadcast(item_genre_vector), on="item_id", how="left")
test_full_features_FULL.count()

2156840

In [114]:
test_full_features.printSchema()

root
 |-- item_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- purchase: string (nullable = true)
 |-- user_buy_frequency: long (nullable = false)
 |-- sum_over_user_id: double (nullable = true)
 |-- one_hot_user_id_solvency: double (nullable = true)
 |-- count_item_id: long (nullable = false)
 |-- sum_over_item_id: double (nullable = true)
 |-- one_hot_item_id_popularity: double (nullable = true)
 |-- all_genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- all_genres_vectors: vector (nullable = true)
 |-- first_genre: string (nullable = true)
 |-- genre_Index: double (nullable = false)
 |-- genre_vector: vector (nullable = true)



In [115]:
test_full_features_FULL.filter(test_full_features_FULL.all_genres.isNull()).count()

19304

In [116]:
test_full_features = test_full_features.na.fill(0,["all_genres"])

In [117]:
test_full_features_NOT_FOUND = test_full_features_FULL.where("all_genres is null")
test_full_features_NOT_FOUND.count()

19304

In [329]:
select_features2 = test_full_features.select('item_id','user_id','one_hot_user_id_solvency','one_hot_item_id_popularity','all_genres_vectors','genre_vector','purchase')\
.withColumn("item_user",f.concat(f.col('item_id'), f.col('user_id')))

select_features2 = select_features2.withColumn("item_id", select_features2["item_id"].cast(IntegerType()))
select_features2 = select_features2.withColumn("user_id", select_features2["user_id"].cast(IntegerType()))
select_features2 = select_features2.withColumn("purchase", select_features2["purchase"].cast(IntegerType()))
select_features2 = select_features2.withColumn("item_user", select_features2["item_user"].cast(DoubleType()))

select_features2.where("user_id=747028").show(1)

+-------+-------+------------------------+--------------------------+--------------------+--------------+--------+----------------+
|item_id|user_id|one_hot_user_id_solvency|one_hot_item_id_popularity|  all_genres_vectors|  genre_vector|purchase|       item_user|
+-------+-------+------------------------+--------------------------+--------------------+--------------+--------+----------------+
| 102279| 747028|     0.18617021276595744|      7.468259895444362E-4|(83,[2,4],[1.0,1.0])|(63,[0],[1.0])|    null|1.02279747028E11|
+-------+-------+------------------------+--------------------------+--------------------+--------------+--------+----------------+
only showing top 1 row



In [330]:
assembler2 = VectorAssembler(inputCols=["item_id", "user_id","item_user", "one_hot_user_id_solvency", "one_hot_item_id_popularity","all_genres_vectors","genre_vector"],outputCol="features") # "all_genres_vectors",
output2 = assembler2.transform(select_features2)

In [333]:
predictions2 = model.transform(output2)

In [334]:
from pyspark.sql import functions as F
predictions2.printSchema()

root
 |-- item_id: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- one_hot_user_id_solvency: double (nullable = true)
 |-- one_hot_item_id_popularity: double (nullable = true)
 |-- all_genres_vectors: vector (nullable = true)
 |-- genre_vector: vector (nullable = true)
 |-- purchase: integer (nullable = true)
 |-- item_user: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [338]:
from pyspark.sql.functions import regexp_replace

predictions2 = predictions2.withColumn("probability", predictions2["probability"].cast("string"))    # First conversion this column data format
predictions2 = predictions2.withColumn('probabilityre',f.split(regexp_replace("probability", "^\[|\]", ""), ",")[1].cast('double'))\
                                             .select('user_id','item_id','probabilityre')\
                                             .withColumnRenamed("probabilityre","probability")   # Extract String Data Format

In [340]:
predictions2.show(10)

+-------+-------+--------------------+
|user_id|item_id|         probability|
+-------+-------+--------------------+
|   1654|  94814|0.043646521427027074|
|   1654|  93629| 0.04364652142697256|
|   1654|   9980| 0.04364652142695613|
|   1654|  95099| 0.04364652142696179|
|   1654|  11265| 0.04364652142701164|
|   1654|  88896| 0.04364652142697156|
|   1654|  67740| 0.04364652142697156|
|   1654|  74271|  0.0436465214270213|
|   1654|  99871| 0.04364652142695591|
|   1654|  78570| 0.04364652142697156|
+-------+-------+--------------------+
only showing top 10 rows



In [341]:
from pyspark.sql import Window
w = Window().orderBy('user_id')
predictions2 = predictions2.withColumn("row_num", F.row_number().over(w))
predictions2.show(10)

+-------+-------+--------------------+-------+
|user_id|item_id|         probability|row_num|
+-------+-------+--------------------+-------+
|   1654|  94814|0.043646521427027074|      1|
|   1654|  93629| 0.04364652142697256|      2|
|   1654|   9980| 0.04364652142695613|      3|
|   1654|  95099| 0.04364652142696179|      4|
|   1654|  11265| 0.04364652142701164|      5|
|   1654|  88896| 0.04364652142697156|      6|
|   1654|  67740| 0.04364652142697156|      7|
|   1654|  74271|  0.0436465214270213|      8|
|   1654|  99871| 0.04364652142695591|      9|
|   1654|  78570| 0.04364652142697156|     10|
+-------+-------+--------------------+-------+
only showing top 10 rows



In [345]:
predictions2 = predictions2.select('row_num','user_id','item_id','probability')
predictions2.show(10)

+-------+-------+-------+--------------------+
|row_num|user_id|item_id|         probability|
+-------+-------+-------+--------------------+
|      1|   1654|  94814|0.043646521427027074|
|      2|   1654|  93629| 0.04364652142697256|
|      3|   1654|   9980| 0.04364652142695613|
|      4|   1654|  95099| 0.04364652142696179|
|      5|   1654|  11265| 0.04364652142701164|
|      6|   1654|  88896| 0.04364652142697156|
|      7|   1654|  67740| 0.04364652142697156|
|      8|   1654|  74271|  0.0436465214270213|
|      9|   1654|  99871| 0.04364652142695591|
|     10|   1654|  78570| 0.04364652142697156|
+-------+-------+-------+--------------------+
only showing top 10 rows



In [1223]:
predictions2.write.format("csv").save("lab03.csv")    

In [1226]:
test_full_features_NOT_FOUND = test_full_features_NOT_FOUND.select('user_id','item_id').repartition(1)
test_full_features_NOT_FOUND.write.format("csv").save("lab03_NOT_FOUND2.csv")  